In [1]:
from sklearn.decomposition import PCA
from tdamapper.core import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.plot import MapperLayoutInteractive
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from tdamapper.clustering import FailSafeClustering
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn import metrics
import pandas as pd
import numpy as np

from functions import *
from chi import *
from regressionP import *
from models import *
from distance import *

06/18/2024 04:14:29 PM utils INFO: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
06/18/2024 04:14:29 PM utils INFO: NumExpr defaulting to 8 threads.


In [2]:
data1 = pd.read_csv("./Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("./Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("./Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("./Data/NPA_TMA2_4.csv", low_memory=False)[:-2]
dataA2 = pd.concat([data1, data2, data3, data4], ignore_index=True)
dataA1 = pd.read_csv("./Data/NPA_TMA1.csv")[:-2]
# Qmap
school_data = pd.read_csv('CalculatedData/coord/school_data.csv')
train_data = pd.read_csv('CalculatedData/coord/train_data.csv')
post_data = pd.read_csv('CalculatedData/coord/post_data.csv')
museum_data = pd.read_csv('CalculatedData/coord/museum_data.csv')
guesthouse_data = pd.read_csv('CalculatedData/coord/guesthouse_data.csv')
themepark_data = pd.read_csv('CalculatedData/coord/themepark_data.csv')
supermarket_data = pd.read_csv('CalculatedData/coord/supermarket_data.csv')
coffee_data = pd.read_csv('CalculatedData/coord/coffee_data.csv')
fastfood_data = pd.read_csv('CalculatedData/coord/fastfood_data.csv')
rtc_data = pd.read_csv('CalculatedData/coord/rtc_data.csv')
thsrc_data = pd.read_csv('CalculatedData/coord/thsrc_data.csv')
baseball_data = pd.read_csv('CalculatedData/coord/baseball_data.csv')
night_data = pd.read_csv('CalculatedData/coord/night_data.csv')
port_data = pd.read_csv('CalculatedData/coord/port_data.csv')
library_data = pd.read_csv('CalculatedData/coord/library_data.csv')
bank_data = pd.read_csv('CalculatedData/coord/bank_data.csv')
hospital_data = pd.read_csv('CalculatedData/coord/hospital_data.csv')
temple_data = pd.read_csv('CalculatedData/coord/temple_data.csv')
police_data = pd.read_csv('CalculatedData/coord/police_data.csv')
gas_data = pd.read_csv('CalculatedData/coord/gas_data.csv')
town_office_data = pd.read_csv('CalculatedData/coord/town_office_data.csv')
hr_office_data = pd.read_csv('CalculatedData/coord/hr_office_data.csv')
mv_data = pd.read_csv('CalculatedData/coord/mv_data.csv')
ntb_data = pd.read_csv('CalculatedData/coord/ntb_data.csv')
# 開放平台
scenic = pd.read_csv("./Data/Scenic_Spot_C_f.csv")

In [3]:
# List of columns to select
select_lst = [
    '天候名稱', 
    '路面狀況-路面狀態名稱',
    '肇因研判大類別名稱-主要', '當事者屬-性-別名稱', '當事者事故發生時年齡', 
    '車輛撞擊部位大類別名稱-最初',
    '光線名稱',
    '道路類別-第1當事者-名稱',
    '速限-第1當事者', 
    '道路型態大類別名稱',
    '事故位置大類別名稱', 
    '號誌-號誌種類名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '事故類型及型態大類別名稱',
    '死亡受傷人數',
    '經度', '緯度',
    '道路型態子類別名稱', '事故位置子類別名稱', '車道劃分設施-分向設施子類別名稱', '事故類型及型態子類別名稱', 
    '當事者行動狀態子類別名稱', '車輛撞擊部位子類別名稱-最初', '車輛撞擊部位子類別名稱-其他', '肇因研判子類別名稱-個別',
    
    '當事者區分-類別-大類別名稱-車種', '當事者區分-類別-子類別名稱-車種',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', '當事者行動狀態大類別名稱',
    '車輛撞擊部位大類別名稱-其他', '肇因研判大類別名稱-個別', '肇事逃逸類別名稱-是否肇逃',
    '路面狀況-路面鋪裝名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱','號誌-號誌動作名稱',
]
def preprocess(input_data, select_lst, sample = 592):
    sample_data = input_data[input_data['當事者順位'] == 1].reset_index(drop=True, inplace=False)
    dataA = sample_data[select_lst]
    
    death_injury_data = split_death_injury(dataA['死亡受傷人數'])
    dist_df = pd.concat([dataA, death_injury_data], axis=1)
    dist_df.drop(columns=['死亡受傷人數'], inplace=True)
    
    return dist_df, sample_data

In [5]:
dist_dfA1 = preprocess(dataA1, select_lst, sample = 592)
dist_dfA2 = preprocess(dataA2, select_lst, sample = 5920)

data_frames = {
    '學校': school_data,
    # '火車站': train_data,
    # '郵局': post_data,
    # '博物館': museum_data,
    # '民宿': guesthouse_data,
    # '遊樂園': themepark_data,
    # # '大賣場': supermarket_data,
    # # '咖啡店': coffee_data,
    # # '速食店': fastfood_data,
    # '捷運車站': rtc_data,
    # '高鐵車站': thsrc_data,
    # # '棒球場': baseball_data,
    # '夜市': night_data,
    # '漁港': port_data,
    # # '圖書館': library_data,
    # # '銀行': bank_data,
    # '醫院': hospital_data,
    # '寺廟': temple_data,
    # '警察局': police_data,
    # '加油站': gas_data,
    # '公所': town_office_data,
    # # '戶政事務所': hr_office_data,
    # '監理站': mv_data,
    # '國稅局': ntb_data,
}
for colname, data_frame in data_frames.items():
    scenic_dfA1 = calculate_distances(dist_dfA1[0], data_frame, colname=colname)
    scenic_dfA2 = calculate_distances(dist_dfA2[0], data_frame, colname=colname)
    
rbind_data = pd.concat([scenic_dfA1, scenic_dfA2], axis=0, ignore_index=True)

for col in data_frames.keys():
    rbind_data.loc[rbind_data[col] > 1, col] = 2
rbind_data.loc[rbind_data['受傷'] > 1, '受傷'] = 2
rbind_data['速限-第1當事者'] = rbind_data['速限-第1當事者'].apply(lambda x: 1 if x > 60 else 0)
rbind_data = process_age(rbind_data)

dist_df = process_data(rbind_data)
scaler = StandardScaler()

full_dist = pd.DataFrame(scaler.fit_transform(dist_df), columns = dist_df.columns)
X1 = full_dist.drop(['受傷', '死亡', '經度', '緯度'], axis=1).to_numpy()

In [21]:
# for i in dist_df.columns:
    # print(dist_df[i].value_counts())

In [6]:
def find_ratio(input_data, components) :
    best_comp = {}
    for comp in range(1,components+1):   
        pca = PCA(comp).fit(input_data)
        
        best_comp[comp] = pca.explained_variance_ratio_.sum()
        
    max_comp = max(best_comp, key=best_comp.get)  # 使用 key=best_comp.get 找到最大值的鍵
    print("最佳成分數：", max_comp)
    print("解釋方差比率累計值：", best_comp[max_comp])

# lens1 = find_ratio(X1, 30)

In [10]:
# lens1 = PCA(10).fit_transform(X1)

# mapper_algo1 = MapperAlgorithm(
#     cover = CubicalCover(
#         n_intervals = 3,
#         overlap_frac = 0.3
#     ),
#     clustering = FailSafeClustering(
#         clustering = AgglomerativeClustering(3, affinity='euclidean', linkage='ward'),
#         verbose = False)
# )

# mapper_graph1 = mapper_algo1.fit_transform(X1, lens1)

In [12]:
import pickle

with open('CalculatedData/mapper_graph1.pkl', 'rb') as f:
    mapper_graph1 = pickle.load(f)
    
mapper_plot1 = MapperLayoutInteractive(
    mapper_graph1,
    colors = dist_df[['速限-第1當事者']].to_numpy(),
    cmap = 'jet',
    # agg = np.nanmean,
    agg = most_frequent_nonan,
    dim = 3,
    iterations = 30,
    seed = 6,
    width = 800,
    height = 500)

# fig_mean1 = mapper_plot1.plot()
# fig_mean1.show(config={'scrollZoom': True})

In [13]:
x = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['x']
y = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['y']
z = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['z']

threeDimData = pd.DataFrame({'x': x, 'y': y, 'z': z})

import re
data_tuple = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['text']

data = []
for item in data_tuple:
    color = int(re.search(r'color: (\d+)', item).group(1))
    node = int(re.search(r'node: (\d+)', item).group(1))
    size = int(re.search(r'size: (\d+)', item).group(1))
    data.append({'color': color, 'node': node, 'size': size})
component_info = pd.DataFrame(data)

full_info = pd.concat([component_info, threeDimData], axis=1)

mp_content_origin = vars(mapper_plot1._MapperLayoutInteractive__graph)['_node']

mp_content = pd.DataFrame.from_dict(mp_content_origin, orient='index')
mp_content.reset_index(inplace=True)
mp_content.rename(columns={'index': 'node'}, inplace=True)

full_info = pd.merge(full_info, mp_content, on=['node', 'size'], how='inner')

In [14]:
# 0.045 預測最好
calinski_data = get_calinski_from_db(full_info, 0.045)

label_0 = full_info[full_info['label'] == 0]
label_1 = full_info[full_info['label'] == 1]
label_2 = full_info[full_info['label'] == 2]
label_out = full_info[(full_info['label'] != 1) & (full_info['label'] != 2) & (full_info['label'] != 0)]

count_0 = get_count_dict(label_0)
count_1 = get_count_dict(label_1)
count_2 = get_count_dict(label_2)
count_out = get_count_dict(label_out)

print(full_info['label'].unique())

len(count_out)

[ 0  1  2 -1]


131

In [15]:
full_0 = rbind_data.loc[count_0.keys()]
full_1 = rbind_data.loc[count_1.keys()]
full_2 = rbind_data.loc[count_2.keys()]

# full_0['weight'] = full_0.index.map(count_0)
# full_1['weight'] = full_1.index.map(count_1)
# full_2['weight'] = full_2.index.map(count_2)

# 離群值不需要被處理
full_out = rbind_data.loc[count_out.keys()]

lst01 = list(count_0.keys() & count_1.keys())
lst02 = list(count_0.keys() & count_2.keys())
lst12 = list(count_1.keys() & count_2.keys())
lsto0 = list(count_out.keys() & count_0.keys())
lsto1 = list(count_out.keys() & count_1.keys())
lsto2 = list(count_out.keys() & count_2.keys())

# 將重複的key另外拉出進行分析，這裡drop是為了符合卡方的獨立性前提假設
full_01 = full_0.loc[lst01]
full_02 = full_0.loc[lst02]
full_12 = full_1.loc[lst12]

full_combine = pd.concat([full_01, full_02, full_12], axis=0) # full_out
full_combine = full_combine.reset_index()
full_combine = full_combine.drop_duplicates(subset='index', keep='first')
full_combine = full_combine.drop('index', axis=1)
# 去掉連接點，使分析更嚴謹
full_0 = full_0.drop(lst01, errors='ignore')
full_0 = full_0.drop(lst02, errors='ignore')
full_0 = full_0.drop(lsto0, errors='ignore')

full_1 = full_1.drop(lst01, errors='ignore')
full_1 = full_1.drop(lst12, errors='ignore')
full_1 = full_1.drop(lsto1, errors='ignore')

full_2 = full_2.drop(lst02, errors='ignore')
full_2 = full_2.drop(lst12, errors='ignore')
full_2 = full_2.drop(lsto2, errors='ignore')

print('01連接點數量', len(lst01))
print('02連接點數量', len(lst02))
print('12連接點數量', len(lst12))
print('o0連接點數量', len(lsto0))
print('o1連接點數量', len(lsto1))
print('o2連接點數量', len(lsto2))
print('離群值數量', full_out.shape[0])

full_combine.shape[0] + full_0.shape[0] + full_1.shape[0] + full_2.shape[0] == rbind_data.shape[0]

01連接點數量 220
02連接點數量 0
12連接點數量 0
o0連接點數量 18
o1連接點數量 11
o2連接點數量 15
離群值數量 131


False

In [78]:
lst_logit = [
    '路面狀況-路面狀態名稱',
    '當事者屬-性-別名稱', '當事者事故發生時年齡', 
    '車輛撞擊部位大類別名稱-最初',
    '光線名稱',
    '道路類別-第1當事者-名稱',
    '速限-第1當事者',
    '道路型態大類別名稱',
    '事故位置大類別名稱', 
    '號誌-號誌種類名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '事故類型及型態大類別名稱',
    '事故位置子類別名稱', '事故類型及型態子類別名稱', 
    '當事者行動狀態子類別名稱',
    '車輛撞擊部位子類別名稱-最初', '肇因研判子類別名稱-個別',
    '當事者區分-類別-子類別名稱-車種',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', 
    '當事者行動狀態大類別名稱',
    '車輛撞擊部位大類別名稱-其他', 
    '肇因研判大類別名稱-個別', '肇事逃逸類別名稱-是否肇逃',
    '路面狀況-路面鋪裝名稱', 
    '車道劃分設施-分向設施子類別名稱',
    '道路障礙-障礙物名稱', '車輛撞擊部位子類別名稱-其他'
    
    # '號誌-號誌動作名稱', '當事者區分-類別-大類別名稱-車種', '肇因研判大類別名稱-主要' # 降低預測值
    # '道路障礙-視距名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車輛撞擊部位大類別名稱-其他' # 降低預測值
    # '道路型態子類別名稱', '路面狀況-路面缺陷名稱', '天候名稱' # 降低
]

def get_clusterN_logit(cluster_data, lst):
    scaler = StandardScaler()
    
    c0_for_lm = process_data(cluster_data)
    c0_for_lm_X = pd.DataFrame(scaler.fit_transform(c0_for_lm), columns=c0_for_lm.columns).reset_index(drop=True, inplace=False)
    # label設定
    c0_for_lm_y = cluster_data.apply(lambda row: 1 if row['死亡'] != 0 else 2, axis=1)
        
    c0_for_lm_X = c0_for_lm_X[lst]
    
    return c0_for_lm_X, c0_for_lm_y

In [79]:
full_combine_X, full_combine_y = get_clusterN_logit(full_combine, lst_logit)
full_0_X, full_0_y = get_clusterN_logit(full_0, lst_logit)
full_1_X, full_1_y = get_clusterN_logit(full_1, lst_logit)
full_2_X, full_2_y = get_clusterN_logit(full_2, lst_logit)

full_out_X, full_out_y = get_clusterN_logit(full_out, lst_logit) # 新增

print(full_combine_y.value_counts())
print(full_0_y.value_counts())
print(full_1_y.value_counts())
print(full_2_y.value_counts())

2    214
1      6
dtype: int64
2    112034
1       440
dtype: int64
2    5659
1      73
dtype: int64
2    2390
1      65
dtype: int64


In [76]:
# 0.045 效果最好
# matrix_combine, score_combine, cm_combine = logistic_cm_gridsearch(full_combine_X,  full_combine_y)
matrix_0, score_0, cm_0 = logistic_cm_gridsearch(full_0_X, full_0_y)
matrix_1, score_1, cm_1 = logistic_cm_gridsearch(full_1_X, full_1_y)
matrix_2, score_2, cm_2 = logistic_cm_gridsearch(full_2_X, full_2_y)
# print(score_combine, score_0, score_1, score_2)
print(score_0, score_1, score_2)

0.7657657657657657 0.8055555555555556 0.6904761904761905


In [77]:
de = full_0_X.shape[0] + full_1_X.shape[0] + full_2_X.shape[0]
logit_avg_score = (full_0_X.shape[0]/de)*score_0 + (full_1_X.shape[0]/de)*score_1 + (full_2_X.shape[0]/de)*score_2
print(logit_avg_score)

0.7661241182387203


In [58]:
de = full_0_X.shape[0] + full_1_X.shape[0] + full_2_X.shape[0]
logit_avg_score = (full_0_X.shape[0]/de)*score_0 + (full_1_X.shape[0]/de)*score_1 + (full_2_X.shape[0]/de)*score_2
print(logit_avg_score)

0.7656396834921793


In [24]:
origin_X, origin_y = get_clusterN_logit(rbind_data, lst_logit)

matrix_origin, score_origin, cm_origin = logistic_cm_gridsearch(origin_X, origin_y)
print(score_origin)

0.6892655367231638


In [23]:
for i in lst_logit:
    print(dist_df[i].value_counts())

0    108994
4     11752
3       114
2       113
1        39
Name: 路面狀況-路面狀態名稱, dtype: int64
3    76427
0    43244
4      885
2      455
1        1
Name: 當事者屬-性-別名稱, dtype: int64
 3    35027
 1    32637
 2    30941
 4    18055
 0     3545
-1      807
Name: 當事者事故發生時年齡, dtype: int64
 1    67227
 2    42073
 0    10458
-1     1254
Name: 車輛撞擊部位大類別名稱-最初, dtype: int64
1    49983
0    42216
2    28813
Name: 光線名稱, dtype: int64
3    96101
5    12223
0     4043
7     2932
6     2380
8     1911
1      985
2      344
4       93
Name: 道路類別-第1當事者-名稱, dtype: int64
0    118484
1      2528
Name: 速限-第1當事者, dtype: int64
0    72496
2    47380
3      693
1      438
4        5
Name: 道路型態大類別名稱, dtype: int64
0    68392
3    50478
2     1923
1      219
Name: 事故位置大類別名稱, dtype: int64
0    71035
1    31739
2    12116
3     6122
Name: 號誌-號誌種類名稱, dtype: int64
2    42605
4    35524
0    24515
3    17604
1      764
Name: 車道劃分設施-分向設施大類別名稱, dtype: int64
0    69254
3    37561
4     8381
1     4296
2     1520
Name: 車道劃分設施